### EXAMPLE OF BASELINE MODELS IN "PREDICTIVE" MODE

"Predictive" mode means we try to predict forest state at time t, with informations from historical field surveys (IFN data at time t-1, t-2, etc...) and, optionnaly, from satellite or meteo data at time t (present time) (as it is described in this [diagram](schematic_diagram_of_predictive_models.png))

It's as if we don't have access to NFI data (because, perhaps, the last census campaign was done five or six years ago, for example...) and we are trying to predict the state of the forest just with the information available.

This could be a "time series" problem, but with the difficulty that we only have three historical data points (when we try to predict LFI4 with LFI1, LFI2 and LFI3).
Classical model for "time series" like Prophet from FB would have a hard time working in this case, because it could be difficult to just observe a target and its periodicity with theses three points.

#### Specific approach for our Baseline Model :

One of first possible approaches is to neutralize the "time series'" problematic, and deal with just the problematic to predict t just with t-1 informations... target at time t will be our target for all the features at t-1. It becomes a classical Machine Learning modelization.

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, RidgeClassifier
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

import tensorflow as tf
import tensorflow_addons as tfa

Import data :

In [3]:
data_base = pd.read_excel('../1_DATA_global_processing/data_processed_and_merged/big_merge_V2_meteo_SAT.xlsx').drop('Unnamed: 0', axis=1)

-------------------

SPECIAL TEMPORAL PREPROCESSING :

Now, we use all the first type of "temporal" preprocessing built for our situation... This code is a extract from the notebook [preprocessing_temporal_correlations.ipynb](preprocessing_temporal_correlations.ipynb) in the parent folder "common_preprocessing_notebookks", a notebook built and shared for all my collaborators.

Feature engineering :

In [4]:
# adding aridity index
data_base["AI"] = data_base['PRCP_GROWTH'] / data_base['TAVE_GROWTH']
# adding H/D index
data_base["H_D"] = data_base['HAUTEUR_ARBRE'] / data_base['DBH']

Target :

In [5]:
TARGET = ['SURF_TER_HA']

Features :

Here we can select features for the past and the future.
The knowledge of the future is only satellite and meteorological data.

In [6]:
# --- PAST --- 
# Logically, we could include the target, because it's a known feature in the past time...
cat_strict = ['PRODREG', 'ESPECE_DOM', 'TYP_RAJ_PPL', 'DEG_FERMETURE', 'STR_PPL', 'RELIEF'] #exemple 'PRODREG', 'ESPECE_DOM', 'TYP_RAJ_PPL', 'DEG_FERMETURE', 'STR_PPL', 'RELIEF'
cat_ord_miss = ['HT_VEG'] #exemple 'TAILLE_PPL', 'MELANGE', 'QUAL_STATION', 'TAUX_COUV_RAJ', 'SURF_TROU_AER', 'HT_VEG'
numerics = ['UNIT_ACCR','H_D','AI','SDI', 'AGE_PPL','ALT', 'TIGES_VIV_H', 'SURF_TER_HA', 'FEUILL_PER', 'CONIF_PER','PERF_CROI', 'NDVI', 'EVI', 'NDMI', 'NDWI', 'DSWI'] #exemple

# --- FUTURE KNOWN ---
add_cat_IFN_stable = [] # technos-substituts : 'DEG_FERMETURE', 'STR_PPL', 'ESPECE_DOM', 'DEG_FERMETURE'
add_cat_ord_IFN_stable = [] # technos-substituts : 'DEG_FERMETURE', 'STR_PPL', 'SURF_TROU_AER' 'SURF_TROU_AER'
add_IFN_numerics_stable = [] # technos-substituts : , 'SDI'
# données potentiellement connues car stables par parcelles :
#['LAT', 'LON', 'ALT', 'PRODREG', 'HT_VEG', 'SLOPE25', 'ASPECT25', 'ORIENTATION', 'PERF_CROI', 'QUAL_STATION', 'UNIT_VEG_FINE',
# 'UNIT_VEG_GROS', 'PROCESS_SILVA', 'PERI_CHENAUX', 'PERI_COULEES','PERI_AVALANCH', 'PERI_CHUTES', 'ETAGE', 'ENDOMMAGEMENT','NB_DEGAT_ARBRE']
# + données extrapolées grâce à des technologies prometteuses (satellites ?)....

add_meteo_known = ['PRCP', 'TAVE_AVG',	'TAVE', 'TAVE_GROWTH', 'PRCP_S_S',	'PRCP_G_S']

add_SAT_known = ['NDVI', 'EVI', 'NDMI', 'NDWI', 'DSWI']

We make two list for the past features and tne future features (it's very hard to say "future feature"... try it !) :

In [7]:
features_past = numerics + cat_strict + cat_ord_miss
features_future = add_cat_IFN_stable + add_cat_ord_IFN_stable + add_IFN_numerics_stable + add_meteo_known + add_SAT_known

We need to split the dataframe in differents LFI parts :

In [8]:
data_LFI1 = data_base.loc[data_base['LFI']=='LFI1',:].sort_values('PARCELLE')
data_LFI2 = data_base.loc[data_base['LFI']=='LFI2',:].sort_values('PARCELLE')
data_LFI3 = data_base.loc[data_base['LFI']=='LFI3',:].sort_values('PARCELLE')
data_LFI4 = data_base.loc[data_base['LFI']=='LFI4',:].sort_values('PARCELLE')

And now we, in each of three first temporal dataframe (LFI1, LFI2, LFI3) we add the future informations (target and know data of LFI t+1 ...)

We use, in parallel, a list system to store all feature names in a global category 'future_feat_names' and also three subcategories ('add_cat_strict_feat_names', 'add...', etc) that will be used for feature extraction later.

In [9]:
future_feat_names = []
add_cat_strict_feat_names = []
add_cat_ord_feat_names = []
add_numerics_feat_names = []

for cat in features_future:
    lfi2_list = data_LFI2[cat].to_list()
    lfi3_list = data_LFI3[cat].to_list()
    lfi4_list = data_LFI4[cat].to_list()
    data_LFI1[cat + "_f"] = lfi2_list
    data_LFI2[cat + "_f"] = lfi3_list
    data_LFI3[cat + "_f"] = lfi4_list
    future_feat_names.append(cat + '_f')
    if cat in add_cat_ord_IFN_stable:
        add_cat_ord_feat_names.append(cat + '_f')
    elif cat in add_cat_IFN_stable:
        add_cat_strict_feat_names.append(cat + '_f')
    else:
        add_numerics_feat_names.append(cat + '_f')


Now, we can concatenate all the different parts of LFI1, LFI2, LFI3 to re-build a unique dataframe.
And also a target with the same concatenation.

In [10]:
data_red = pd.concat([data_LFI1, data_LFI2, data_LFI3], axis=0)[features_past + future_feat_names]
Y = pd.concat([data_LFI2[TARGET], data_LFI3[TARGET], data_LFI4[TARGET]], axis = 0)

In [11]:
data_red.head()

,UNIT_ACCR,H_D,AI,SDI,AGE_PPL,ALT,TIGES_VIV_H,SURF_TER_HA,FEUILL_PER,CONIF_PER,...,TAVE_AVG_f,TAVE_f,TAVE_GROWTH_f,PRCP_S_S_f,PRCP_G_S_f,NDVI_f,EVI_f,NDMI_f,NDWI_f,DSWI_f
0,NaN,0.695652,NaN,571,85.0,715.918970,590.0,27.79,NaN,NaN,...,8.8859,9.026806,13.84776,117.278617,51.835496,0.4992,0.0160,0.2705,-0.4653,0.6865
5,NaN,0.647727,NaN,890,140.0,563.829759,400.0,53.38,NaN,NaN,...,8.9951,19.793490,13.83959,122.675652,55.827235,0.5552,0.0149,0.2017,-0.4864,0.6996
9,NaN,0.666667,NaN,489,80.0,564.885846,320.0,26.70,NaN,NaN,...,9.0637,11.671380,13.39843,128.067334,61.309991,0.5633,0.0153,0.1727,-0.5002,0.6852
14,NaN,NaN,NaN,0,2.0,563.551602,0.0,0.00,NaN,NaN,...,8.8805,10.459870,13.15530,130.759873,63.219754,0.6059,0.0197,0.2269,-0.5588,0.8384
19,NaN,0.676471,NaN,377,130.0,539.769096,150.0,23.32,NaN,NaN,...,8.8384,11.544840,13.16893,131.143585,62.702111,0.6012,0.0184,0.2029,-0.5486,0.7902


We also need to transform our target in a simple list :

In [12]:
Y = Y.to_numpy().ravel()

------------------

USUAL PREPROCESSING :

This part is similar to the preprocessing of the "descriptive" mode.

Preprocessing for other ordered categorical feature ("-1" class management like it is done in the "descriptive" mode):

In [13]:
feats_cat_ord = cat_ord_miss + add_cat_ord_feat_names
for cat in feats_cat_ord:
  data_red[cat] = data_red[cat].apply(lambda v : v if v!=-1 else np.nan)

Splitting :

In [14]:
X_train, X_test, y_train, y_test = train_test_split(data_red, Y, test_size=0.2, random_state=2)

PREPROCESSING PIPELINE :

In [15]:
numerics_features = numerics + feats_cat_ord + add_numerics_feat_names
feats_cat_strict = cat_strict + add_cat_strict_feat_names

In [16]:
numerics_transforms = Pipeline(
    [('imputer',KNNImputer()),
    ('encoder',StandardScaler())
])
categorials_transforms = Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder(drop="first"))
])

preprocessor = ColumnTransformer(
    [("num", numerics_transforms, numerics_features),
     ("cat", categorials_transforms, feats_cat_strict)])

In [17]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

c:\Users\PC\anaconda3\lib\site-packages\sklearn\impute\_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)


In [23]:
np.shape(X_train)

(5767, 67)

In [24]:
np.shape(X_test)

(1442, 67)

-------------------------

BASELINE MODEL 1 : Ridge Regression

Model Definition and GrideSearch definition :

In [18]:
model =  Ridge(max_iter=10000)

In [19]:
params = {
    'alpha':[45, 46, 47]
}

Here, we have selected the final hyperparameters, after a back and forth in the parameters and the training result...

In [20]:
grid = GridSearchCV(model, param_grid=params, scoring='r2', verbose=2)

Training :

In [21]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] END ...........................................alpha=45; total time=   0.0s
[CV] END ...........................................alpha=45; total time=   0.0s
[CV] END ...........................................alpha=45; total time=   0.0s
[CV] END ...........................................alpha=45; total time=   0.0s
[CV] END ...........................................alpha=45; total time=   0.0s
[CV] END ...........................................alpha=46; total time=   0.0s
[CV] END ...........................................alpha=46; total time=   0.0s
[CV] END ...........................................alpha=46; total time=   0.0s
[CV] END ...........................................alpha=46; total time=   0.0s
[CV] END ...........................................alpha=46; total time=   0.0s
[CV] END ...........................................alpha=47; total time=   0.0s
[CV] END ........................................

GridSearchCV(estimator=Ridge(max_iter=10000),
             param_grid={'alpha': [45, 46, 47]}, scoring='r2', verbose=2)

In [22]:
grid.best_estimator_

Ridge(alpha=46, max_iter=10000)

Scores :

In [23]:
train_scores = cross_val_score(grid.best_estimator_, X_train, y_train, cv=5)
test_scores = cross_val_score(grid.best_estimator_, X_test, y_test, cv=5)
print(f'Train score mean : {np.mean(train_scores)}')
print(f'Train score std : {np.std(train_scores)}')
print(f'Test score mean : {np.mean(test_scores)}')
print(f'Test score std : {np.std(test_scores)}')

Train score mean : 0.6582697534344525
Train score std : 0.037993051662840546
Test score mean : 0.69053424369463
Test score std : 0.03651874264200527


Ok, it's a not very good prediction...

For information, a collaborator (Arnaud) had fun creating simple models that predicted for the target, either all the time the "average" trend of the historical target at t-1, or either a constant stable system (target at t = target at t-1)... R2 scores were respectively 0,68 and 0,63... All theses scores are present in our official dashboard.


So, it's not very interesting to use a model in Machine Learning with a big computing time (and energy consumption !) to do the same thing we could do with just a pen and a paper.

Let's see the feature extraction :

In [24]:
list_features_in = []
for feat in numerics_features:
  list_features_in.append(feat)
for cat in feats_cat_strict:
  nb_lab = len(data_red[cat].unique())-1
  for i in range(nb_lab):
    list_features_in.append(f'{cat}_{i}')

In [25]:
df_coef = pd.DataFrame(grid.best_estimator_.coef_, columns=['Coeff'], index= list_features_in)

In [26]:
fig = px.bar(df_coef['Coeff'], title=f"Features importance for target : {TARGET} with Lasso Linear Regression")
fig.show()

As we can imagine, this simple Ridge regressor model essentially relies on historical target data to predict the current time target.
"SDI" play also...
And we could be a little enjoyed, because satellite data are presents... especially the spectral bands function "DSWI" that measures the "water stress" of the vegetation. This feature plays as a future feature, and also as a past feature.

-----------------

BASELINE MODEL 2 : Simple Deep Learning Regressor

We try with a more complex model in deep learning... As it is a strutured data context, pehaps we won't get any good results... but perhaps there are non-linear or complex relationship between our features, so...

DATASETS CONSTITUTION :

In [27]:
train_batch = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=len(X_train)).batch(batch_size=32)
test_batch = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size=32)

In [28]:
np.shape(X_train)

(5767, 67)

In [29]:
for x, y in train_batch.take(1):
    print(y)

tf.Tensor(
[52.09  4.65 69.68 42.19 32.09 33.24 39.43 33.48 16.71 42.92 33.94 67.15
 37.88 36.69 18.88  1.43 45.14 91.43 13.19 56.43 30.4  36.07 17.7  66.72
 21.09 24.72 56.4  57.03 36.08 23.56 47.02 63.08], shape=(32,), dtype=float64)


Our model is a very simple Multi-LAyers Dense, with 32 / 16 / 8 neurons units... and finally a unique neuron for the linear prediction

In [39]:
model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(np.shape(X_train)[1],)),
        tf.keras.layers.Dense(64,"relu"),
        tf.keras.layers.Dense(32,"relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16,"relu"),
        tf.keras.layers.Dense(8,"relu"),
        tf.keras.layers.Dense(1, 'linear')
    ])

Compile (with metrics and loss function for a linear regression) :

In [40]:
model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=tfa.metrics.RSquare())

Training :

In [41]:
model.fit(train_batch, epochs=20, validation_data=test_batch)

Epoch 1/20
181/181 [==============================] - 3s 12ms/step - loss: 505.0738 - r_square: -0.2463 - val_loss: 137.4863 - val_r_square: 0.6221
Epoch 2/20
181/181 [==============================] - 2s 11ms/step - loss: 172.6847 - r_square: 0.5739 - val_loss: 124.7412 - val_r_square: 0.6572
Epoch 3/20
181/181 [==============================] - 2s 11ms/step - loss: 163.1366 - r_square: 0.5975 - val_loss: 120.6369 - val_r_square: 0.6684
Epoch 4/20
181/181 [==============================] - 2s 11ms/step - loss: 159.7096 - r_square: 0.6059 - val_loss: 118.8692 - val_r_square: 0.6733
Epoch 5/20
181/181 [==============================] - 2s 11ms/step - loss: 156.6747 - r_square: 0.6134 - val_loss: 115.7333 - val_r_square: 0.6819
Epoch 6/20
181/181 [==============================] - 2s 11ms/step - loss: 152.0760 - r_square: 0.6248 - val_loss: 113.9270 - val_r_square: 0.6869
Epoch 7/20
181/181 [==============================] - 2s 11ms/step - loss: 151.6275 - r_square: 0.6259 - val_loss: 12

Score on test set :

In [42]:
r2_score(y_test,model.predict(X_test))

46/46 [==============================] - 0s 4ms/step


0.6881676595692697

Ok, it's not a very good prediction, too...

Let's see the feature extraction :

To do that, a simply approch is to take the mean of all the weights of the first layer  related to a feature, and keeping this value for every feature...

In [44]:
coeff_mean = []
for i in range(np.shape(X_train)[1]):
    coeff_mean.append(np.mean(model.layers[0].trainable_variables[0][i]))


In [45]:
df_coef = pd.DataFrame(coeff_mean, columns=['Coeff'], index= list_features_in)

In [46]:
fig = px.bar(df_coef['Coeff'], title=f"Features importance for target : {TARGET} with Lasso Linear Regression")
fig.show()

Once again, several historical data from field survey (IFN) are most important :
- 'SURF_TER_HA', our target at time t-1 (it's logical)
- 'ESPECE_DOM', dominant species of the forest
- 'H_D', Rapport of the height and the diameter of all tree
- 'RELIEF'...

And our satellite or meteo data have a very few importance...